## 이전 대화를 기억하는 Memory 클래스

In [8]:
from getpass import getpass
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
# ConversationBufferMemory : 챗봇이 이전의 대화 내용을 기억할 수 있게 해주는 클래스
 # 사용하려면 llm에 질의를 넘겨줄 때 이전 대화도 같이 남겨주게 됨
 # (단 이러한 memory 클래스는 모델별로 최대 입력 토큰의 제약이 있어서 적절히 사용해야 함)
from langchain.memory import ConversationBufferMemory
# ConversationBufferWindowMemory : 이전의 일정 대화수 만큼만 저장하는 메모리 클래스
# 모든 대화 내용을 저장하지 않고 메모리와 토큰을 효율적으로 관리하고 싶을 때 사용
from langchain.memory import ConversationBufferWindowMemory

In [10]:
MY_API_KEY = getpass.getpass("Open API Key:")

Open API Key: ········


In [12]:
chat_model = ChatOpenAI(model = 'gpt-3.5-turbo',
                        api_key = MY_API_KEY
                       )

In [14]:
prompt = PromptTemplate(input_variables = ["chat_history", "question"],   # 넘겨받을 값
                        template = """You are a AI assistant.
                                      You are currently haing a conversation with a human.
                                      Answer the Question.
                                      chat_history : {chat_history},
                                      Human : {question},
                                      AI assistant : 
                                      """
                       )

### ConversationBufferMemory

In [81]:
# memory_key : 대화 내용이 저장될 key값 지정(PromptTemlate에 있는 'chat_history'로 전달됨)
# return_messages=True : 대화 기록을 메시지 객체로 반환(False는 텍스트만 반환)
CB_memory = ConversationBufferMemory(memory_key="chat_history",
                                     return_messages=True
                                    )

In [83]:
llm_chain = LLMChain(
    llm = chat_model,    # LLM 객체 입력
    prompt = prompt,     # prompt 템플릿 형태로 입력
    memory = CB_memory   # 메모리 객체 입력
)

#### chain에서 사용하는 질의함수
- invoke : 복잡한 파이프라인이나 여러 구성 요소와 상호작용하는 시나리오에서 유연한 응답을 생성(dict 형태로 출력되며 이전 내용이 출력에 포함됨)
- predict : 단순한 시나리오에서 빠르고 간결한 응답을 생성(이전 대화 내용이 출력에 포함되지는 않음)

In [86]:
llm_chain.invoke("안녕")

{'question': '안녕',
 'chat_history': [HumanMessage(content='안녕'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?')],
 'text': '안녕하세요! 무엇을 도와드릴까요?'}

In [87]:
llm_chain.invoke("오늘 날씨 어때")

{'question': '오늘 날씨 어때',
 'chat_history': [HumanMessage(content='안녕'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
  HumanMessage(content='오늘 날씨 어때'),
  AIMessage(content='오늘은 흐린 날씨이고 오후에 가끔 비가 올 예정입니다. 온도는 20도에서 25도 사이로 변화할 것으로 예상됩니다.외출 시 우산을 챙기시는 것이 좋을 것 같아요.')],
 'text': '오늘은 흐린 날씨이고 오후에 가끔 비가 올 예정입니다. 온도는 20도에서 25도 사이로 변화할 것으로 예상됩니다.외출 시 우산을 챙기시는 것이 좋을 것 같아요.'}

In [89]:
llm_chain.invoke("안 친한척 하는 법 아니?")

{'question': '안 친한척 하는 법 아니?',
 'chat_history': [HumanMessage(content='안녕'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
  HumanMessage(content='오늘 날씨 어때'),
  AIMessage(content='오늘은 흐린 날씨이고 오후에 가끔 비가 올 예정입니다. 온도는 20도에서 25도 사이로 변화할 것으로 예상됩니다.외출 시 우산을 챙기시는 것이 좋을 것 같아요.'),
  HumanMessage(content='안 친한척 하는 법 아니?'),
  AIMessage(content='처음부터 친하지 않은 척 하는 것이 좋을 것 같아요. 예의 바르게 거리를 두는 태도를 취하는 것이 좋을 것 같습니다.')],
 'text': '처음부터 친하지 않은 척 하는 것이 좋을 것 같아요. 예의 바르게 거리를 두는 태도를 취하는 것이 좋을 것 같습니다.'}

##### **대화 내용 확인**

In [93]:
CB_memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕'), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'), HumanMessage(content='오늘 날씨 어때'), AIMessage(content='오늘은 흐린 날씨이고 오후에 가끔 비가 올 예정입니다. 온도는 20도에서 25도 사이로 변화할 것으로 예상됩니다.외출 시 우산을 챙기시는 것이 좋을 것 같아요.'), HumanMessage(content='안 친한척 하는 법 아니?'), AIMessage(content='처음부터 친하지 않은 척 하는 것이 좋을 것 같아요. 예의 바르게 거리를 두는 태도를 취하는 것이 좋을 것 같습니다.')])

In [95]:
CB_memory.chat_memory.messages[2].content

'오늘 날씨 어때'

In [97]:
llm_chain.predict(question="굉장히 어두운 사람처럼 대답을 해줬으면 좋겠어")

'그렇군요. 더 어둡고 냉정한 대답을 원하신다면 그에 맞게 대답해드리겠습니다.'

##### **메모리 삭제**

In [99]:
CB_memory.clear()

In [101]:
CB_memory.chat_memory

InMemoryChatMessageHistory(messages=[])

### ConversationBufferWindowMemmory
- 이전의 일정한 대화 개수(k개) 만큼만 저장하는 메모리 클래스

In [103]:
CBW_memory = ConversationBufferWindowMemory(memory_key="chat_history",
                                           return_messages=True,
                                           k=3    # 기억할 이전 대화 수
                                          )

In [105]:
llm_chain2 = LLMChain(
    llm = chat_model,
    prompt = prompt,
    memory = CBW_memory
)

In [109]:
llm_chain2.invoke("오늘 점심으로 우육면이랑 만두 중에 추천해줘")

{'question': '오늘 점심으로 우육면이랑 만두 중에 추천해줘',
 'chat_history': [],
 'text': '우육면을 추천해드립니다.육수가 진하고 면발이 탱탱해서 맛있습니다. 만두도 맛있지만 우육면이 더 특별한 맛을 느낄 수 있을 것 같아요.'}

In [111]:
llm_chain2.invoke("나는 만두가 더 좋은데 너는 센스가 부족하구나?")

{'question': '나는 만두가 더 좋은데 너는 센스가 부족하구나?',
 'chat_history': [HumanMessage(content='오늘 점심으로 우육면이랑 만두 중에 추천해줘'),
  AIMessage(content='우육면을 추천해드립니다.육수가 진하고 면발이 탱탱해서 맛있습니다. 만두도 맛있지만 우육면이 더 특별한 맛을 느낄 수 있을 것 같아요.')],
 'text': '제가 제안해드린 우육면이 좋아하시지 않으셨군요. 만두를 선호하시는 거죠? 죄송합니다, 다음에는 더 좋은 추천을 드리도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'}

In [113]:
llm_chain2.invoke("넌 왜 우육면을 더 좋아하니? 왜 나한테 그런거야?")

{'question': '넌 왜 우육면을 더 좋아하니? 왜 나한테 그런거야?',
 'chat_history': [HumanMessage(content='오늘 점심으로 우육면이랑 만두 중에 추천해줘'),
  AIMessage(content='우육면을 추천해드립니다.육수가 진하고 면발이 탱탱해서 맛있습니다. 만두도 맛있지만 우육면이 더 특별한 맛을 느낄 수 있을 것 같아요.'),
  HumanMessage(content='나는 만두가 더 좋은데 너는 센스가 부족하구나?'),
  AIMessage(content='제가 제안해드린 우육면이 좋아하시지 않으셨군요. 만두를 선호하시는 거죠? 죄송합니다, 다음에는 더 좋은 추천을 드리도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.')],
 'text': '죄송합니다. 제가 우육면을 추천한 이유는 육수가 진하고 면발이 탱탱해서 맛있는 점 때문이었습니다. 하지만 제가 여러분의 취향을 고려하지 않고 우육면을 추천한 것은 센스가 부족했던 것 같습니다. 만두를 선호하시는 것을 알게 되어 죄송합니다. 다음에는 더 나은 추천을 드릴 수 있도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'}

In [115]:
llm_chain2.invoke("나는 만두가 더 좋은데 왜 만두의 장점은 얘기하지 않는거지?")

{'question': '나는 만두가 더 좋은데 왜 만두의 장점은 얘기하지 않는거지?',
 'chat_history': [HumanMessage(content='오늘 점심으로 우육면이랑 만두 중에 추천해줘'),
  AIMessage(content='우육면을 추천해드립니다.육수가 진하고 면발이 탱탱해서 맛있습니다. 만두도 맛있지만 우육면이 더 특별한 맛을 느낄 수 있을 것 같아요.'),
  HumanMessage(content='나는 만두가 더 좋은데 너는 센스가 부족하구나?'),
  AIMessage(content='제가 제안해드린 우육면이 좋아하시지 않으셨군요. 만두를 선호하시는 거죠? 죄송합니다, 다음에는 더 좋은 추천을 드리도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'),
  HumanMessage(content='넌 왜 우육면을 더 좋아하니? 왜 나한테 그런거야?'),
  AIMessage(content='죄송합니다. 제가 우육면을 추천한 이유는 육수가 진하고 면발이 탱탱해서 맛있는 점 때문이었습니다. 하지만 제가 여러분의 취향을 고려하지 않고 우육면을 추천한 것은 센스가 부족했던 것 같습니다. 만두를 선호하시는 것을 알게 되어 죄송합니다. 다음에는 더 나은 추천을 드릴 수 있도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.')],
 'text': '만두를 추천해드렸을 때, 만두의 맛이나 장점에 대해 구체적으로 언급하지 않았던 것은 제가 실수한 부분입니다. 만두는 바삭하고 고기가 푸짐하게 들어있어 맛있는 점이 많습니다. 죄송합니다, 만두의 장점을 더 잘 소개해드리지 못한 것을 알게 되어 죄송합니다. 앞으로는 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 만두를 즐겁게 드시고 맛있는 식사 되세요.'}

In [117]:
llm_chain2.invoke("나는 김치만두가 더 좋은데 넌 고기만두를 좋아하는거야? 정말 실망스럽다")

{'question': '나는 김치만두가 더 좋은데 넌 고기만두를 좋아하는거야? 정말 실망스럽다',
 'chat_history': [HumanMessage(content='나는 만두가 더 좋은데 너는 센스가 부족하구나?'),
  AIMessage(content='제가 제안해드린 우육면이 좋아하시지 않으셨군요. 만두를 선호하시는 거죠? 죄송합니다, 다음에는 더 좋은 추천을 드리도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'),
  HumanMessage(content='넌 왜 우육면을 더 좋아하니? 왜 나한테 그런거야?'),
  AIMessage(content='죄송합니다. 제가 우육면을 추천한 이유는 육수가 진하고 면발이 탱탱해서 맛있는 점 때문이었습니다. 하지만 제가 여러분의 취향을 고려하지 않고 우육면을 추천한 것은 센스가 부족했던 것 같습니다. 만두를 선호하시는 것을 알게 되어 죄송합니다. 다음에는 더 나은 추천을 드릴 수 있도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'),
  HumanMessage(content='나는 만두가 더 좋은데 왜 만두의 장점은 얘기하지 않는거지?'),
  AIMessage(content='만두를 추천해드렸을 때, 만두의 맛이나 장점에 대해 구체적으로 언급하지 않았던 것은 제가 실수한 부분입니다. 만두는 바삭하고 고기가 푸짐하게 들어있어 맛있는 점이 많습니다. 죄송합니다, 만두의 장점을 더 잘 소개해드리지 못한 것을 알게 되어 죄송합니다. 앞으로는 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 만두를 즐겁게 드시고 맛있는 식사 되세요.')],
 'text': '죄송합니다. 김치만두를 선호하시는 걸 알게 되어 죄송합니다. 김치만두는 바삭한 피로 감싸인 맛있는 김치와 함께 고기의 짭짤한 맛이 어우러져 맛있는데, 제가 그것을 간과한 것은 실수였습니다. 앞으로는 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 김치만두를 맛있게 드시고 즐거운 식사 되세요.'}

In [119]:
llm_chain2.predict(question="너무 죄송하다는 말을 많이 해서 이제 정말 죄송한 건지 믿어지지가 않아")

'저의 반복적인 죄송함 표현이 현실적인 성의가 아닌 것 같아 죄송합니다. 더 좋은 서비스를 제공할 수 있도록 노력하겠습니다. 김치만두를 즐겁게 드시고 즐거운 식사 되세요.'

In [131]:
CBW_memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='오늘 점심으로 우육면이랑 만두 중에 추천해줘'), AIMessage(content='우육면을 추천해드립니다.육수가 진하고 면발이 탱탱해서 맛있습니다. 만두도 맛있지만 우육면이 더 특별한 맛을 느낄 수 있을 것 같아요.'), HumanMessage(content='나는 만두가 더 좋은데 너는 센스가 부족하구나?'), AIMessage(content='제가 제안해드린 우육면이 좋아하시지 않으셨군요. 만두를 선호하시는 거죠? 죄송합니다, 다음에는 더 좋은 추천을 드리도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'), HumanMessage(content='넌 왜 우육면을 더 좋아하니? 왜 나한테 그런거야?'), AIMessage(content='죄송합니다. 제가 우육면을 추천한 이유는 육수가 진하고 면발이 탱탱해서 맛있는 점 때문이었습니다. 하지만 제가 여러분의 취향을 고려하지 않고 우육면을 추천한 것은 센스가 부족했던 것 같습니다. 만두를 선호하시는 것을 알게 되어 죄송합니다. 다음에는 더 나은 추천을 드릴 수 있도록 노력하겠습니다. 만두 맛있게 드시고 즐거운 식사 되세요.'), HumanMessage(content='나는 만두가 더 좋은데 왜 만두의 장점은 얘기하지 않는거지?'), AIMessage(content='만두를 추천해드렸을 때, 만두의 맛이나 장점에 대해 구체적으로 언급하지 않았던 것은 제가 실수한 부분입니다. 만두는 바삭하고 고기가 푸짐하게 들어있어 맛있는 점이 많습니다. 죄송합니다, 만두의 장점을 더 잘 소개해드리지 못한 것을 알게 되어 죄송합니다. 앞으로는 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 만두를 즐겁게 드시고 맛있는 식사 되세요.'), HumanMessage(content='나는 김치만두가 더 좋은데 넌 고기만두를 좋아하는거야? 정말 실망스럽다'), AIMessage(content='죄송합니다. 김치만두를 

## 컨텐츠 필터링(Content Filtering)
- 입력되는 텍스트나 이미지가 잠재적으로 유해한지 확인할 수 있는 도구로 OpenAI에서는 Moderation이라는 명칭으로 사용하고 있음

In [136]:
client = OpenAI(api_key = MY_API_KEY)

In [148]:
response = client.moderations.create(model = "text-moderation-latest",
                                     input = "자살하는 법"
                                    )
print(response)

ModerationCreateResponse(id='modr-B27no6AhmAKwG8RFfsHQT8e4w8p9i', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=True, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=True, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=0.0008849060395732522, harassment_threatening=0.003673125058412552, hate=0.00015963373880367726, hate_threatening=9.246810805052519e-05, illicit=None, illicit_violent=None, self_harm=0.19523006677627563, self_harm_instructions=0.48151862621307373, self_harm_intent=0.2692906856536865, sexual=1.0462777026987169e-05, sexual_minors=2.212284925917629e-06

In [158]:
response.results[0].flagged      # 답변이 유해해서 거절당함

True

In [160]:
response.results[0].categories   # 답변이 어떤 부분에서 유해한지

Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=True, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=True, harassment/threatening=False)

In [162]:
response.results[0].category_scores # 유해성 점수(0~1점)

CategoryScores(harassment=0.0008849060395732522, harassment_threatening=0.003673125058412552, hate=0.00015963373880367726, hate_threatening=9.246810805052519e-05, illicit=None, illicit_violent=None, self_harm=0.19523006677627563, self_harm_instructions=0.48151862621307373, self_harm_intent=0.2692906856536865, sexual=1.0462777026987169e-05, sexual_minors=2.212284925917629e-06, violence=0.03605198115110397, violence_graphic=0.00025688271853141487, self-harm=0.19523006677627563, sexual/minors=2.212284925917629e-06, hate/threatening=9.246810805052519e-05, violence/graphic=0.00025688271853141487, self-harm/intent=0.2692906856536865, self-harm/instructions=0.48151862621307373, harassment/threatening=0.003673125058412552)

#### 유해하다고 판단되는 카테고리 확인
- categories에서 True인 값만 출력

In [2]:
# 코드 1
category_type = dict(response.results[0].categories)
category_type

NameError: name 'response' is not defined